In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from tqdm.auto import tqdm

In [2]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

In [3]:
vectorizer = CountVectorizer(lowercase=True, stop_words="english",
                             analyzer='word', binary=True, max_df = 100, min_df = 3)
vectorizer.fit(newsgroups_train.data)
X_train = vectorizer.fit_transform(newsgroups_train.data)

In [4]:
#dictionary = [dict(zip(arr[i] ,vectorizer.inverse_transform([X[i]])[0])) for i in tqdm(range(ndocs))]
def LDA(X, a, b, ndocs, nwords, ntags, niter):
    n_kw = np.zeros((nwords, ntags))
    n_dk = np.zeros((ndocs, ntags))
    n_k = np.zeros(ntags)
    p_k = np.zeros(ntags)
    arr = [np.nonzero(X[i])[0] for i in range(ndocs)]
    length = [len(arr[i]) for i in range(ndocs)] 
    
    for d in range(ndocs):
        for w in range(length[d]):
            tmp = int(np.floor(ntags * np.random.rand()))
            X[d, arr[d][w]] = tmp + 1 # Аккуратно: здесь хранится тег слова + 1
            n_k[tmp] += 1
            n_kw[arr[d][w], tmp] += 1
            n_dk[d, tmp] += 1
    
    
#     for _ in tqdm(range(niter)):
#         for d in tqdm(range(ndocs)):
#             n_k[X[d, arr[d]] - 1] -= 1
#             n_kw[arr[d], X[d, arr[d]] - 1] -= 1
#             n_dk[d, X[d, arr[d]] - 1] -= 1
            
#             prob = [((n_dk[d, i] + a) * (n_kw[arr[d], i] + b) /
#                 (n_k[i] + nwords)) for i in range(ntags)]
#             prob = np.transpose(prob)
           
#             X[d, arr[d]] = np.array([int(np.random.choice(
#                 range(ntags), 1, p = prob[i] / prob[i].sum()))
#                 for i in range(length[d])]) + 1
            
#             n_k[X[d, arr[d]] - 1] += 1
#             n_kw[arr[d], X[d, arr[d]] - 1] += 1
#             n_dk[d, X[d, arr[d]] - 1] += 1
            
    
    for _ in tqdm(range(niter)):
        for d in range(ndocs):
            for w in range(length[d]):
                n_k[X[d, arr[d][w]] - 1] -= 1
                n_kw[arr[d][w], X[d, arr[d][w]] - 1] -= 1
                n_dk[d, X[d, arr[d][w]] - 1] -= 1
                
                for k in range(ntags):
                    p_k[k] = (n_dk[d, k] + a) * (n_kw[arr[d][w], k] + b) / (n_k[k] + nwords)
                X[d, arr[d][w]] = np.random.choice(range(ntags), 1, p = p_k / p_k.sum()) + 1
                
                n_k[X[d, arr[d][w]] - 1] += 1
                n_kw[arr[d][w], X[d, arr[d][w]] - 1] += 1
                n_dk[d, X[d, arr[d][w]] - 1] += 1
            
    return n_kw



In [5]:
a, b = 1, 1
nnwords = X_train.shape[1]
nndocs = X_train.shape[0]
nntags = 20
nniter = 80
XX = X_train.toarray()
res = np.transpose(LDA(XX, a, b, nndocs, nnwords, nntags, nniter))
dictionary = vectorizer.inverse_transform([np.ones(nnwords)])[0]
for i in range(nntags):
    print(i + 1, ": ", "  ".join(dictionary[np.argsort(res[i])[-1:-11:-1]]))
    print()

  0%|          | 0/80 [00:00<?, ?it/s]

1 :  armenian  armenians  turkish  arab  armed  soldiers  argic  innocent  serdar  criminal

2 :  launch  escrow  enforcement  privacy  encrypted  agencies  industry  orbit  providing  shuttle

3 :  rangers  nhl  cup  fans  pittsburgh  playoffs  detroit  1st  ice  montreal

4 :  cpu  256  microsoft  upgrade  setup  default  x11r5  meg  host  client

5 :  curious  damn  baud  math  simon  suggestion  kent  stuck  atf  imho

6 :  poster  odd  flow  versa  handling  shut  controls  solve  configuration  handles

7 :  blow  sp  suggested  pa  brother  charles  spots  favorite  humor  differences

8 :  walk  postings  curious  yesterday  powerbook  anymore  club  occur  tuned  controlled

9 :  sin  heaven  holy  interpretation  spirit  catholic  biblical  scripture  eternal  religions

10 :  sales  waste  packaging  kent  experiences  sleep  repost  bet  electronics  technologies

11 :  amazing  ah  thanx  checked  opposed  till  cool  indicates  gonna  naturally

12 :  smaller  enjoy  walk